In [ ]:
# Otto, parallel cross validation
from __future__ import print_function
import pandas as pd
from xgboost import XGBClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import LabelEncoder
import time
from matplotlib import pyplot

In [24]:
# load data
data = pd.read_csv('train.csv')
dataset = data.values
# split data into X and y
X = dataset[:,0:94]
y = dataset[:,94]

In [25]:
# let's see what it looks like
print(data.shape)
data.head()

(61878, 95)


id  feat_1  feat_2  feat_3  feat_4  feat_5  feat_6  feat_7  feat_8  feat_9  \
0   1       1       0       0       0       0       0       0       0       0   
1   2       0       0       0       0       0       0       0       1       0   
2   3       0       0       0       0       0       0       0       1       0   
3   4       1       0       0       1       6       1       5       0       0   
4   5       0       0       0       0       0       0       0       0       0   

    ...     feat_85  feat_86  feat_87  feat_88  feat_89  feat_90  feat_91  \
0   ...           1        0        0        0        0        0        0   
1   ...           0        0        0        0        0        0        0   
2   ...           0        0        0        0        0        0        0   
3   ...           0        1        2        0        0        0        0   
4   ...           1        0        0        0        0        1        0   

   feat_92  feat_93   target  
0        0        0  Class_1  
1        0        0  Class_1  
2        0        0  Class_1  
3        0        0  Class_1  
4        0        0  Class_1  

[5 rows x 95 columns]

In [34]:
# pretty boring.  let's check ranges and NaN
with pd.option_context('display.max_rows', 999, 'display.max_columns', 999):
    print(data.describe())

                 id       feat_1        feat_2        feat_3        feat_4  \
count  61878.000000  61878.00000  61878.000000  61878.000000  61878.000000   
mean   30939.500000      0.38668      0.263066      0.901467      0.779081   
std    17862.784315      1.52533      1.252073      2.934818      2.788005   
min        1.000000      0.00000      0.000000      0.000000      0.000000   
25%    15470.250000      0.00000      0.000000      0.000000      0.000000   
50%    30939.500000      0.00000      0.000000      0.000000      0.000000   
75%    46408.750000      0.00000      0.000000      0.000000      0.000000   
max    61878.000000     61.00000     51.000000     64.000000     70.000000   

             feat_5        feat_6        feat_7        feat_8        feat_9  \
count  61878.000000  61878.000000  61878.000000  61878.000000  61878.000000   
mean       0.071043      0.025696      0.193704      0.662433      1.011296   
std        0.438902      0.215333      1.030102      2.25577

In [19]:
# looks like there's a chicken in every pot
pots = data.isnull().sum()
for pot, chicken in pots.iteritems():
    print(pot, chicken)

id 0
feat_1 0
feat_2 0
feat_3 0
feat_4 0
feat_5 0
feat_6 0
feat_7 0
feat_8 0
feat_9 0
feat_10 0
feat_11 0
feat_12 0
feat_13 0
feat_14 0
feat_15 0
feat_16 0
feat_17 0
feat_18 0
feat_19 0
feat_20 0
feat_21 0
feat_22 0
feat_23 0
feat_24 0
feat_25 0
feat_26 0
feat_27 0
feat_28 0
feat_29 0
feat_30 0
feat_31 0
feat_32 0
feat_33 0
feat_34 0
feat_35 0
feat_36 0
feat_37 0
feat_38 0
feat_39 0
feat_40 0
feat_41 0
feat_42 0
feat_43 0
feat_44 0
feat_45 0
feat_46 0
feat_47 0
feat_48 0
feat_49 0
feat_50 0
feat_51 0
feat_52 0
feat_53 0
feat_54 0
feat_55 0
feat_56 0
feat_57 0
feat_58 0
feat_59 0
feat_60 0
feat_61 0
feat_62 0
feat_63 0
feat_64 0
feat_65 0
feat_66 0
feat_67 0
feat_68 0
feat_69 0
feat_70 0
feat_71 0
feat_72 0
feat_73 0
feat_74 0
feat_75 0
feat_76 0
feat_77 0
feat_78 0
feat_79 0
feat_80 0
feat_81 0
feat_82 0
feat_83 0
feat_84 0
feat_85 0
feat_86 0
feat_87 0
feat_88 0
feat_89 0
feat_90 0
feat_91 0
feat_92 0
feat_93 0
target 0


In [35]:
# encode string class values as integers
label_encoded_y = LabelEncoder().fit_transform(y)
# prepare cross validation
kfold = StratifiedKFold(label_encoded_y, n_folds=10, shuffle=True, random_state=7)

In [47]:
data['target'] = label_encoded_y
covariance = data.cov()
sortedCovs = sorted([(feat, val) for feat, val in covariance.target.iteritems()], key = lambda x: abs(x[1]), reverse=True)
for x in sortedCovs:
    print(x)

('id', 43824.699694555318)
('target', 6.3040842227512446)
('feat_14', -2.8996243883181543)
('feat_40', -2.735034891798342)
('feat_24', 2.6375663369766471)
('feat_15', -2.512054719379361)
('feat_67', 2.4962669009480458)
('feat_90', 2.2561691966759998)
('feat_69', 2.2480464175071253)
('feat_54', 1.8948716038551721)
('feat_58', 1.7445803330088283)
('feat_25', -1.7226146610681645)
('feat_75', 1.6579315325334569)
('feat_19', 1.5312857863214637)
('feat_9', -1.5298542609863004)
('feat_76', 1.461802333412779)
('feat_36', 1.4533973987527811)
('feat_11', 1.4479206417928394)
('feat_62', 1.304521242985216)
('feat_88', -1.2773135048836439)
('feat_72', -1.2633823254966479)
('feat_64', -1.1943808173091524)
('feat_8', 1.1878354113528282)
('feat_60', 1.1694651604560704)
('feat_35', 1.1646016835484503)
('feat_73', 1.1467634997993956)
('feat_3', 1.0959438238687713)
('feat_4', 1.0742233514340955)
('feat_59', 1.0353801021952498)
('feat_83', 1.0138014769327901)
('feat_13', 0.98235900572782664)
('feat_80', 0

In [36]:
# Single Thread XGBoost, Parallel Thread CV
start = time.time()
model = XGBClassifier(nthread=1)
results = cross_val_score(model, X, label_encoded_y, cv=kfold, scoring='log_loss', n_jobs=-1)
elapsed = time.time() - start
print("Single Thread XGBoost, Parallel Thread CV: %f" % (elapsed))

KeyboardInterrupt: 

In [ ]:
# Parallel Thread XGBoost, Single Thread CV
start = time.time()
model = XGBClassifier(nthread=-1)
results = cross_val_score(model, X, label_encoded_y, cv=kfold, scoring='log_loss', n_jobs=1)
elapsed = time.time() - start
print("Parallel Thread XGBoost, Single Thread CV: %f" % (elapsed))

In [ ]:
# Parallel Thread XGBoost and CV
start = time.time()
model = XGBClassifier(nthread=-1)
results = cross_val_score(model, X, label_encoded_y, cv=kfold, scoring='log_loss', n_jobs=-1)
elapsed = time.time() - start
print("Parallel Thread XGBoost and CV: %f" % (elapsed))